In [1]:
cd ..

/home/hyunjun/workspace/technical_indicator


In [2]:
import talib as ta
import pandas as pd

from coinmarketcap_dataloader import CoinMarketCap
from coinmarketcap_dataloader import CMC_API_KEY

### Indicator Module

- for loop을 계속 돌리면 api error가 발생해서 time sleep을 넣음 <br>
- 전일까지의 historical data 일부는 멤버변수에 할당해놓고 사용하는 형태로 구현 <br>
- 각 지표의 기간별 값을 ticker key를 갖는 dict으로 리턴: value_dict <br>
- 각 지표의 기간별 스코어를 ticker key를 갖는 dict으로 리턴: score_dict <br> 

In [242]:
import time 
import numpy as np
import talib as ta
import multiprocessing

from tqdm import tqdm
from typing import Tuple
from datetime import datetime
from datetime import timedelta


class Indicator(CoinMarketCap):
    PERIODS = [
        7, 28, 91]

    SCORE_NAMES = [
        '7_period_score', 
        '28_period_score', 
        '91_period_score', 
        'mean_score'
        ]

    INDICATOR_NAMES = [
        'RSI', 'MACD', 
        'BBANDS', 'STOCH', 'OBV']


    def __init__(self, API_KEY):
        super().__init__(API_KEY)

        self.tickers:list = None
        self.historical_data_dict:dict = {}
        self.now_data_dict:dict = {}

        self.update_tickers()

    def get_total_score(self, tickers:list) -> Tuple[dict, dict]:
        """
        입력 ticker들에 대한 total score list를 리턴

        [value_dict]
        {'BTC': {'RSI': [53., 65., 62.],
                'MACD': [1317., 3107., 3222.],
                'BBANDS': [22., 88., 95.],
                'STOCH': [-2., 1., 1.],
                'OBV': [71681935600., 71681935600., 71681935600.]},

        'USDT': {'RSI': [51., 50., 50.],
                'MACD': [-5.6, 1.5, -4.],
                'BBANDS': [58., 53., 49.],
                'STOCH': [0., 0., 0.],
                'OBV': [-610460727736., -610460727736., -610460727736.]},

        'ETH': {'RSI': [50., 60., 57.],
                'MACD': [61., 138., 85.3873846637475],
                'BBANDS': [23., 85., 91.],
                'STOCH': [-1., 0., 1.],
                'OBV': [102378389016.81, 102378389016.81, 102378389016.81]}}
        

        [score_dict]
        {'BTC': {'7_period_score': 1.0,
                '28_period_score': 4.0,
                '91_period_score': 4.0,
                'mean_score': 3.0},

        'USDT': {'7_period_score': -2.0,
                '28_period_score': 0.0,
                '91_period_score': -2.0,
                'mean_score': -1.333},

        'ETH': {'7_period_score': 1.0,
                '28_period_score': 3.0,
                '91_period_score': 4.0,
                'mean_score': 2.667}}
        """

    
        tickers = set(tickers).intersection(
            set(self.historical_data_dict.keys())
        )

        scoring_func = [
            self.get_RSI_score,
            self.get_MACD_score,
            self.get_STOCH_score,
            self.get_BBANDS_score,
            self.get_OBV_score,
        ]

        # realtime dictionary update
        self._get_price_to_now(tickers, types=['open', 'high', 'low', 'close', 'volume'])

        # 모든 ticker의 지표별, 기간별 indicator 값에 대한 어레이
        value_array = [self.get_ALL(ticker) for ticker in tqdm(tickers)]

        # ticker : {indicator: [values1, values2, ...]}
        value_dict = {ticker: {name:value for name, value in zip(self.INDICATOR_NAMES, values)} \
            for ticker, values in zip(tickers, value_array)}

        # 모든 ticker의 지표별, 기간별 score 값에 대한 어레이
        score_array = np.array([ [func(values) for func, values in zip(scoring_func, value_dict[ticker].values())] \
            for ticker in tickers ])

        # 모든 지표에 대해서 합하여 기간별 score 산출
        score_array_sum = np.sum(score_array, axis=1)
        # 기간별 score와 기간별 score의 평균을 묶어서 최종 score 배열로 
        score_array_total = np.concatenate([score_array_sum, np.mean(score_array_sum, axis=-1)[:,np.newaxis]], axis=1)
        score_array_total = np.round(score_array_total, decimals=3)

        # ticker: {period: score}
        score_dict = {ticker: {name:score for name, score in zip(self.SCORE_NAMES, score_list)} \
            for ticker, score_list in zip(tickers, score_array_total)}
    
        return value_dict, score_dict


    def get_ALL(self, ticker:str) -> list:
        """
        모든 지표값을 리턴
        """

        # 사용할 지표 계산 메서드
        all_indicators = [self.get_RSI, 
                          self.get_MACD, 
                          self.get_BBANDS, 
                          self.get_STOCH, 
                          self.get_OBV]
        
        # hitorcial_df, realtime_df update 
        ohlcv = self.now_data_dict[ticker]
        return [func(ohlcv) for func in all_indicators]


    def get_RSI(self, ohlcv:pd.DataFrame) -> list:
        """
        Period별 RSI
        """
        return [ta.RSI(ohlcv['close'].to_numpy(), n)[-1] for n in self.PERIODS]

    
    def get_MACD(self, ohlcv:pd.DataFrame) -> list:
        """
        Period별 MACD 
        """
        return [ta.MACD(ohlcv['close'].to_numpy(), fastperiod=n, slowperiod=int((26/12)*n), signalperiod=int((9/12)*n))[0][-1] for n in self.PERIODS]


    def get_BBANDS(self, ohlcv:pd.DataFrame) -> list:
        """
        Period별 BBANDS 
        """
        return [(ohlcv['close'][-1] - ta.SMA(ohlcv['close'].to_numpy(), timeperiod=n)[-1]) / \
         (ta.STDDEV(ohlcv['close'].to_numpy(), timeperiod=n)[-1] + 1e-5) for n in self.PERIODS]

    
    def get_STOCH(self, ohlcv:pd.DataFrame) -> list:
        """
        Period별 STOCH
        """
        return [ta.STOCH(ohlcv['high'].to_numpy(), ohlcv['low'].to_numpy(), ohlcv['close'].to_numpy(), 
                         fastk_period=n, slowk_period=int((3/14)*n), slowd_period=int((3/14)*n))[0][-1] for n in self.PERIODS]
    

    def get_OBV(self, ohlcv:pd.DataFrame) -> list:
        """
        Period별 OBV
        """
        return [ta.OBV(ohlcv['close'].to_numpy(), ohlcv['volume'].to_numpy())[-1] for n in self.PERIODS]


    def get_RSI_score(self, rsi_values:list):
        """
        RSI 지표 스코어 기준
        """
        def score_func(rsi):
            if rsi < 30: 
                return -1
            elif rsi > 70:
                return 1
            else: 
                return 0
            
        return list(map(score_func, rsi_values))
    

    def get_MACD_score(self, macd_values:list):
        """ 
        MACD 지표 스코어 기준
        """
        def score_func(macd):
            if macd > 0: 
                return 1
            else:
                return -1
            
        return list(map(score_func, macd_values))
        

    def get_BBANDS_score(self, distance_values:list):
        """
        BBANDS 지표 스코어 기준
        """
        def score_func(distance):
            if distance < -1:
                return -1
            elif distance > 1:
                return 1
            else:
                return 0
        
        return list(map(score_func, distance_values))


    def get_STOCH_score(self, slowk_values:list):
        """
        STOCH 지표 스코어 기준
        """
        def score_func(slowk):
            if slowk < 20:
                return -1
            elif slowk > 80:
                return 1
            else:
                return 0
            
        return list(map(score_func, slowk_values))
    

    def get_OBV_score(self, obv_values:list):
        """ 
        OBV 지표 스코어 기준
        """
        def score_func(obv):
            if obv > 0:
                return 1
            else:
                return -1
            
        return list(map(score_func, obv_values))


    def update_tickers(self):
        """
        Ticker List를 멤버변수 self.tickers에 저장
        """
        ticker_infos = self.get_all_listings()
        tickers = [info['symbol'] for info in ticker_infos]
        self.tickers = list( map(lambda x:x.upper(), tickers) )


    def update_historical_data_frame(self, tickers:list=None):
        """
        Ticker 과거 데이터를 멤버변수 self.historical_data_frame에 저장
        """

        tickers = self.tickers if tickers is None else tickers

        for ticker in tqdm(tickers):
            
            try: 
                # time.sleep(0.5)
                self.historical_data_dict.update({ticker:self.get_ohlcv_n(ticker)})
            except:
                try:
                    time.sleep(3.0)
                    self.historical_data_dict.update({ticker:self.get_ohlcv_n(ticker)})
                except:
                    continue


    def get_ohlcv_n(self, ticker:str, n=500) -> pd.DataFrame:
        """ 
        현재 시점으로부터 과거 n개의 ohlcv 데이터를 리턴
        """

        date_ago = datetime.now() - timedelta(days = n + 1)
        historical_info = self.get_ohlcv_historical(ticker, start=date_ago)
        dataframe = pd.DataFrame([info['quote']['USD'] for info in historical_info])
        dataframe = dataframe.set_index('timestamp', drop=True)
        return dataframe
    
        
    def _get_price_to_now(self, tickers:list, types:list = ['close']):

        try:
            realtime_for_all = self.get_ohlcv_realtime(tickers)
        except:
            print('API CALL RETRY')
            realtime_for_all = self.get_ohlcv_realtime(tickers)
        
        # realtime df dictionary update
        for ticker in tickers:
            historical_df = self.historical_data_dict[ticker]
            realtime_df = pd.DataFrame([realtime_for_all[ticker][0]['quote']['USD']])
            realtime_df.set_index('last_updated', inplace=True)
            total_df = pd.concat([historical_df[types], realtime_df[types]])
            self.now_data_dict[ticker] = total_df


    def _get_normalize(self, array:np.array, upper:int=5, lower:int=-5):
        """
        Min-Max normalize으로 upper, lower bounding
        """
        k = upper - lower
        d = -lower

        new_array = array.copy()
        max_score = np.max(new_array)
        min_score = np.min(new_array)

        if len(array) == 1:
            return new_array
        
        if max_score == min_score:
            return new_array
    
        new_array = k * (new_array - min_score) / (max_score - min_score) - d
        return new_array

In [243]:
indicator = Indicator(CMC_API_KEY)

In [244]:
tickers = indicator.tickers

#### 전일까지의 과거 데이터 ticker 별로 멤버변수에 저장

In [394]:
ticker = 'BAL'

In [395]:
indicator.update_historical_data_frame([ticker])

100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


#### 기간별 지표값 가져오기

In [396]:
ohlcv = indicator.historical_data_dict[ticker]
len(ohlcv)

500

In [366]:
indicator.get_BBANDS(ohlcv)

[-0.342518814453496, 0.8140857417466382, 1.3053824787202173]

In [367]:
indicator.get_RSI(ohlcv)

[55.32027599426399, 55.127078927327375, 54.01762667952787]

In [368]:
indicator.get_MACD(ohlcv)

[0.07894425935592153, 0.22923854289940482, 0.11503863595616526]

In [369]:
indicator.get_RSI(ohlcv)

[55.32027599426399, 55.127078927327375, 54.01762667952787]

In [370]:
indicator.get_STOCH(ohlcv)

[33.04424086729678, 42.32064143858755, 46.581762654327456]

In [371]:
indicator.get_OBV(ohlcv)

[-9764658.340000032, -9764658.340000032, -9764658.340000032]

### 기간별 score 가져오기

In [372]:
indicator.get_RSI_score(indicator.get_RSI(ohlcv))

[0, 0, 0]

In [373]:
indicator.get_MACD_score(indicator.get_MACD(ohlcv))

[1, 1, 1]

In [374]:
indicator.get_STOCH_score(indicator.get_STOCH(ohlcv))

[0, 0, 0]

In [375]:
indicator.get_BBANDS_score(indicator.get_BBANDS(ohlcv))

[0, 0, 1]

In [376]:
indicator.get_OBV_score(indicator.get_OBV(ohlcv))

[-1, -1, -1]

In [377]:
indicator.get_total_score([ticker])

100%|██████████| 1/1 [00:00<00:00, 879.49it/s]


({'CTX': {'RSI': [56.742548857360745, 55.52301167320017, 54.202972310772005],
   'MACD': [0.08056691693709306, 0.23114839784889152, 0.121352590541419],
   'BBANDS': [0.48052816238833757, 0.9106233670823358, 1.3329263095734853],
   'STOCH': [24.950503472915265, 42.57928587972678, 47.030084344155746],
   'OBV': [-9363379.580000032, -9363379.580000032, -9363379.580000032]}},
 {'CTX': {'7_period_score': 0.0,
   '28_period_score': 0.0,
   '91_period_score': 0.0,
   'mean_score': 0.0}})

#### ticker 여러개를 받아서 지표값, 스코어 받기

In [332]:
value_dict, score_dict = indicator.get_total_score(tickers)

100%|██████████| 7/7 [00:00<00:00, 2801.00it/s]


In [21]:
len(value_dict.keys()), len(score_dict.keys())

(50, 50)